In [50]:
import pandas as pd
import numpy as np
import scipy.stats
import warnings

warnings.filterwarnings("ignore")
gams_python_path="/Library/Frameworks/GAMS.framework/Resources/apifiles/Python/gams"

### Merge old and new fixed Effects

In [107]:
####import msa identifier data 
msa=pd.read_stata('/Users/hannahkamen/Documents/population-migration2/dta/msa_identifier.dta')

####import msa 2019 to 2000 lookup
msa_lookup=pd.read_excel('/Users/hannahkamen/Downloads/msa_2019_lookup_adj.xlsx')

###import fixed effect data
new_fe=pd.read_excel('/Users/hannahkamen/Documents/population-migration2/dta/regionfe_bpl_28_age1_adj.xlsx')
new_fe=msa.merge(new_fe,on='msa',how='inner').merge(msa_lookup,left_on='metarea',right_on='metarea_19',how='inner')
new_fe=new_fe.rename(columns={'regionfe':'regionfe_age1_new'})

for i in np.arange(2,5,1):
    df=pd.read_excel('/Users/hannahkamen/Documents/population-migration2/dta/regionfe_bpl_28_age%s_adj.xlsx'%i)
    df=df.rename(columns={'regionfe':'regionfe_age%s_new'%i})
    new_fe=new_fe.merge(df,on='msa',how='inner')
###limit columns in new fes
new_fe=new_fe[['metarea','metarea_07_lookup','regionfe_age1_new','regionfe_age2_new','regionfe_age3_new','regionfe_age4_new']]
    
####import old fixed effects
old_fe=pd.read_csv('/Users/hannahkamen/Downloads/FE_acs0610.csv')
old_fe=old_fe.rename(columns={'metarea':'metarea_07_lookup','regionfe_age1':'regionfe_age1_old','regionfe_age2':'regionfe_age2_old','regionfe_age3':'regionfe_age3_old','regionfe_age4':'regionfe_age4_old'})
old_fe=old_fe[['metarea_07_lookup','regionfe_age1_old','regionfe_age2_old','regionfe_age3_old','regionfe_age4_old']]


####merge old fe onto new fes
old_new_fe=new_fe.merge(old_fe,on='metarea_07_lookup',how='inner')


###import puma lookup
puma_lkup=pd.read_stata('/Users/hannahkamen/Documents/population-migration2/dta/msa2013_puma2000.dta')

###import albuoy data
albuoy=pd.read_stata('/Users/hannahkamen/Documents/population-migration2/dta/albouy_all.dta')


In [ ]:
# old_fe[old_fe['metarea_07_lookup']=='996']

In [102]:
# old_new_fe[old_new_fe['_merge']!='both']['metarea_07_lookup'].unique()

In [115]:
old_new_fe['metarea']

0                              akron, oh
1            albany-schenectady-troy, ny
2                        albuquerque, nm
3      allentown-bethlehem-easton, pa-nj
4                           amarillo, tx
                     ...                
204                           yakima, wa
205                     york-hanover, pa
206    youngstown-warren-boardman, oh-pa
207                        yuba city, ca
208                             yuma, az
Name: metarea, Length: 209, dtype: object

In [116]:
albuoy_new['metarea']

0                              akron, oh
1            albany-schenectady-troy, ny
2                        albuquerque, nm
3      allentown-bethlehem-easton, pa-nj
4                           amarillo, tx
                     ...                
252                   wy virtual msa, wy
253                           yakima, wa
254                     york-hanover, pa
255                        yuba city, ca
256                             yuma, az
Name: metarea, Length: 257, dtype: object

In [118]:
old_new_fe_m=old_new_fe.merge(albuoy_new,on='metarea',how='inner')

In [120]:
for i in np.arange(1,5,1):
    old_new_fe_m['fe_absdiff_%s'%i]=abs(old_new_fe_m['regionfe_age%s_new'%i]-old_new_fe_m['regionfe_age%s_old'%i])
    

In [122]:
old_new_fe_m.to_excel('/Users/hannahkamen/Downloads/fe_compare.xlsx')

In [ ]:
old_new_fe_m['']

In [123]:
old_new_fe_m

,metarea,metarea_07_lookup,regionfe_age1_new,regionfe_age2_new,regionfe_age3_new,regionfe_age4_new,regionfe_age1_old,regionfe_age2_old,regionfe_age3_old,regionfe_age4_old,hot_new,cold_new,avgtemp_new,fe_absdiff_1,fe_absdiff_2,fe_absdiff_3,fe_absdiff_4
0,"akron, oh","akron, oh",0.000000,0.000000,0.000000,0.000000,1.300557,1.416568,1.691586,1.716620,0.763600,66.986031,50.157440,1.300557,1.416568,1.691586,1.716620
1,"albany-schenectady-troy, ny","albany-schenectady-troy, ny",-0.909092,-0.865428,-0.813570,-0.733166,1.571242,2.002352,1.892351,1.953546,0.169282,90.411346,46.321091,2.480334,2.867780,2.705921,2.686712
2,"albuquerque, nm","albuquerque, nm",1.530907,1.926288,1.953497,2.066950,2.342537,2.742465,2.887812,2.921669,0.521268,39.494156,52.164108,0.811630,0.816177,0.934315,0.854719
3,"allentown-bethlehem-easton, pa-nj","allentown-bethlehem-easton, pa/nj",-0.285636,-0.102270,0.061255,0.203393,0.893502,1.203027,1.355257,1.553075,0.930552,60.634953,50.391243,1.179139,1.305297,1.294002,1.349682
4,"amarillo, tx","amarillo, tx",0.441839,0.347399,0.470762,0.869411,0.446458,0.510540,0.857845,0.903681,10.007876,25.110722,57.315117,0.004620,0.163142,0.387083,0.034270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,"wilmington, nc","wilmington, nc",0.613534,0.555607,0.955738,1.293471,1.101340,1.291536,1.268706,1.364346,13.947460,5.832448,62.476761,0.487806,0.735929,0.312967,0.070875
203,"yakima, wa","yakima, wa",-0.471457,-0.343085,-0.242170,0.136100,0.434924,0.905256,1.204687,1.031152,0.581633,55.660210,47.841599,0.906381,1.248340,1.446857,0.895052
204,"york-hanover, pa","york, pa",-0.384713,-0.104123,-0.082252,0.185958,0.696773,0.926506,1.249674,1.290198,1.372702,50.368160,52.250175,1.081486,1.030630,1.331927,1.104240
205,"yuba city, ca","yuba city, ca",-0.333956,-0.336051,-0.098176,0.282487,-0.457709,-0.303127,-0.034970,0.071707,11.896394,0.907087,60.782570,0.123754,0.032923,0.063207,0.210780


### Merge old and new weather variables

In [110]:
albuoy_old=albuoy=pd.read_stata('/Users/hannahkamen/Documents/population-migration2/acs5yr0610/dta/climate_hetero.dta')
albuoy_old['metarea']=albuoy['metarea'].str.replace("-","/")
albuoy_old['avgtemp_old']=albuoy_old['avgtemp']
albuoy_old['hot_old']=albuoy_old['exthot_28']
albuoy_old['cold_old']=albuoy_old['extcold']
albuoy_old=albuoy_old[['metarea','hot_old','cold_old','avgtemp_old']]


albuoy_new=albuoy=pd.read_stata('/Users/hannahkamen/Documents/population-migration2/dta/albuoy_new.dta')
albuoy_new['avgtemp_new']=albuoy_new['avgtemp']
albuoy_new['hot_new']=albuoy_new['hot']
albuoy_new['cold_new']=albuoy_new['cold']
albuoy_new=albuoy_new[['metarea','hot_new','cold_new','avgtemp_new']]




In [46]:
albuoy_m=albuoy_old.merge(msa_lookup,left_on='metarea',right_on='metarea_07_lookup',how='inner').merge(albuoy_new,left_on='metarea_19',right_on='metarea',how='inner')

albuoy_m['abs_diff_hot']=abs(albuoy_m['hot_new']-albuoy_m['hot_old'])
albuoy_m['abs_diff_cold']=abs(albuoy_m['cold_new']-albuoy_m['cold_old'])
albuoy_m['abs_diff_avgtemp']=abs(albuoy_m['avgtemp_new']-albuoy_m['avgtemp_old'])


In [48]:
albuoy_m.to_excel('/Users/hannahkamen/Downloads/old_new_albuoy_compare.xlsx')